In [1]:
import findspark
findspark.init()

import pyspark
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("Cost Percentage per Environment").getOrCreate()
spark

In [2]:
path = "C:\\Users\\vaish\\pyspark prac\\Data 16-11-n/"

In [3]:
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [4]:
schm = StructType([
    StructField("agmbgm",StringType(),True),
    StructField("business_unit",StringType(),True),
    StructField("dateTime",TimestampType(),True),
    StructField("chargetype",StringType(),True),
    StructField("cluster_cost",BooleanType(),True),
    StructField("cost",DoubleType(),True),
    StructField("cost_location",StringType(),True),
    StructField("currency",StringType(),True),
    StructField("departmentCode",StringType(),True),
    StructField("environment",StringType(),True),
    StructField("exec_owner",StringType(),True),
    StructField("finance_owner",StringType(),True),
    StructField("friendly_name",StringType(),True),
    StructField("is_cie_project",BooleanType(),True),
    StructField("meter",StringType(),True),
    StructField("meter_subcategory",StringType(),True),
    StructField("pricingmodel",StringType(),True),
    StructField("product",StringType(),True),
    StructField("product_owner",StringType(),True),
    StructField("project_name",StringType(),True),
    StructField("res_name",StringType(),True),
    StructField("resource_group",StringType(),True),
    StructField("run_date",StringType(),True),
    StructField("service_name",StringType(),True),
    StructField("service_tier",StringType(),True),
    StructField("sleeve",StringType(),True),
    StructField("unique_id",LongType(),True),
    StructField("to_date",IntegerType(),True),
    StructField("to_hour",IntegerType(),True)
    
    
])

reading non rep data saved in previous task  

In [5]:
df = spark.read.csv(path+'part*',schema = schm)

new dataframe for env and total cost

In [9]:
df_cost = df.groupBy("environment").agg(sum('cost').alias('total_cost')).orderBy('environment')


joining both df

In [10]:
df_req = df.join(df_cost,['environment'],"left")

req percentage cost of project name per environment

In [11]:
df_per = df_req.withColumn("cost_percent",df_req.cost/df_req.total_cost*100)

In [33]:
df_per.filter(df_per.cluster_cost==False).filter(df_per.environment!="None").filter(df_per.project_name!="None").groupBy("environment","project_name").agg(sum('cost_percent').alias('percentage_cost_env')).orderBy("environment","project_name").toPandas()

,environment,project_name,percentage_cost_env
0,Australia Prod AustraliaCentral,Analytics-Services,2.413052e-07
1,Australia Prod AustraliaCentral,BackupPTT,1.053433e+00
2,Australia Prod AustraliaCentral,CAD-WebClient,1.105886e-01
3,Australia Prod AustraliaCentral,CCAdmin-V1,2.151237e+00
4,Australia Prod AustraliaCentral,CCInterfaces,2.800014e-04
...,...,...,...
769,US Government Stage USGovVirginia,kodiak-api-services,1.592814e-06
770,US Government Stage USGovVirginia,media-for-evidence,2.351486e-02
771,US Government Stage USGovVirginia,ngcp,2.245755e-04
772,US Government Stage USGovVirginia,ngdp,1.336160e-07


# ##############